## Testing reading of files
Generated by the leakage simulator developed by JSI from DELFT code

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
tmp_1 = pd.read_csv("simulated_sensor_data_8_cols.csv")

In [ ]:
print(f"Dataframe is of length {len(tmp_1.index)}")
tmp_1.head()

## Determining leak step from duplicates

In [ ]:
# duplicate check
duplicates = tmp_1[tmp_1[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak', 'leak_amount']].duplicated()]
display(duplicates)
unique_leaks = duplicates["leak_amount"].str.replace("LPS", "").unique().astype(float)
# display(sorted(list(unique_leaks)))

In [ ]:
# leak of 0.002 seems to still be noticable on the foruth decimal of pressure

In [ ]:
# dropp the columns containing info about leak and leak amount 
reduced_df = tmp_1[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak']]
reduced_df = reduced_df.round(3)
display(reduced_df)

# drop the rows that aren't unique
no_duplicates_df = reduced_df.drop_duplicates()
display(no_duplicates_df)

In [ ]:
merged_leak = pd.merge(no_duplicates_df, tmp_1["leak_amount"], left_index=True, right_index=True)
merged_leak["leak"] = merged_leak["leak_amount"].str.replace("LPS", "").astype(float)
# display(merged_leak)

# there 127 unique nodes in the leak network
# print(len(merged_leak["node_with_leak"].unique()))

nodes_with_leaks_df = merged_leak[["leak", "node_with_leak"]].groupby("node_with_leak").agg({"leak": list})
# display(nodes_with_leaks_df)
tmp_1["leak"] = tmp_1["leak_amount"].str.replace("LPS", "").astype(float)
main_df_grouped = tmp_1[["leak", "node_with_leak"]].groupby("node_with_leak")\
                    .agg({"leak": list})
# display(main_df_grouped)
display(len(list(main_df_grouped.loc["Node_J-RN1"].values)[0]))

avg = 0
for index, row in main_df_grouped.iterrows():
    avg += len(row["leak"])
    
print(avg)
avg = avg / 127
# there should be 22 601 leakage scenarios generated for every node
# in practice there is 22 648 scenarios, of which 47 are duplicated 
# but here the average of unduplicated values is 1677
print(f"Average number of leaks on each node is {avg}")

In [ ]:
all_leak_vals = sorted(list(main_df_grouped.loc["Node_J-RN1"].values)[0])
import plotly.express as px

#print(len([i for i in range(500, 23101, 1)]))
fig = px.scatter(y=all_leak_vals, x=[i for i in range(len(all_leak_vals))])
fig.show()

## Reducing leak step to remove values which are linearly interpolated

In [ ]:
# TODO remove data points